In [2]:
%pwd

'c:\\Users\\asus\\Documents\\Projects\\ECommerce Customer Churn Prediction\\notebooks'

In [2]:
import os
os.chdir("../")

In [3]:
%pwd

'c:\\Users\\asus\\Documents\\Projects\\ECommerce Customer Churn Prediction'

In [4]:
import pandas as pd
df=pd.read_csv("data/interim/kaggle/raw_kaggle_churn_data.csv")

In [5]:
df.head(5)

,CustomerID,Churn,Tenure,PreferredLoginDevice,CityTier,WarehouseToHome,PreferredPaymentMode,Gender,HourSpendOnApp,NumberOfDeviceRegistered,...,MaritalStatus,NumberOfAddress,Complain,OrderAmountHikeFromlastYear,CouponUsed,OrderCount,DaySinceLastOrder,CashbackAmount,source,data_last_updated_dt
0,50001,1,4.0,Mobile Phone,3,6.0,Debit Card,Female,3.0,3,...,Single,9,1,11.0,1.0,1.0,5.0,159.93,kaggle,2025-02-23 13:01:41
1,50002,1,NaN,Phone,1,8.0,UPI,Male,3.0,4,...,Single,7,1,15.0,0.0,1.0,0.0,120.90,kaggle,2025-02-23 13:01:41
2,50003,1,NaN,Phone,1,30.0,Debit Card,Male,2.0,4,...,Single,6,1,14.0,0.0,1.0,3.0,120.28,kaggle,2025-02-23 13:01:41
3,50004,1,0.0,Phone,3,15.0,Debit Card,Male,2.0,4,...,Single,8,0,23.0,0.0,1.0,3.0,134.07,kaggle,2025-02-23 13:01:41
4,50005,1,0.0,Phone,1,12.0,CC,Male,NaN,3,...,Single,3,0,11.0,1.0,1.0,3.0,129.60,kaggle,2025-02-23 13:01:41


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2815 entries, 0 to 2814
Data columns (total 22 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   CustomerID                   2815 non-null   int64  
 1   Churn                        2815 non-null   int64  
 2   Tenure                       2551 non-null   float64
 3   PreferredLoginDevice         2815 non-null   object 
 4   CityTier                     2815 non-null   int64  
 5   WarehouseToHome              2564 non-null   float64
 6   PreferredPaymentMode         2815 non-null   object 
 7   Gender                       2815 non-null   object 
 8   HourSpendOnApp               2560 non-null   float64
 9   NumberOfDeviceRegistered     2815 non-null   int64  
 10  PreferedOrderCat             2815 non-null   object 
 11  SatisfactionScore            2815 non-null   int64  
 12  MaritalStatus                2815 non-null   object 
 13  NumberOfAddress   

In [10]:
from src.constants import CONFIG_PATH,PARAMS_PATH,SCHEMA_PATH
from src.utils.common import read_yaml
from src.entity.data_ingestion import DataIngestionConfig
from src.entity.data_storage import DataStorageConnectionConfig
from src.entity.data_validation import DataValidationConfig




class ConfigurationManager:
    def __init__(self,config_path=CONFIG_PATH,params_path=PARAMS_PATH,schema_path=SCHEMA_PATH):
        self.config=read_yaml(config_path),
        self.params=read_yaml(params_path),
        self.schema=read_yaml(schema_path)
    def get_data_ingestion_config(self)->DataIngestionConfig:
        config=self.config[0].data
        data_ingestion=DataIngestionConfig(
            kaggle_dataset=config.kaggle_source_path,
            gdrive_dataset=config.gdrive_source_path,
            local_dataset=config.raw,
            staging_dataset=config.interim
        )
        return data_ingestion
    def get_data_storage_config(self)->DataStorageConnectionConfig:
        config=self.config[0].sql
        data_connection=DataStorageConnectionConfig(
            server_name=config.server_name,
            db_name=config.db_name,
            driver_name=config.driver_name,
            trusted_conn=config.trusted_conn,
            kaggle_table=config.kaggle_table_name,
            gdrive_table=config.gdrive_table_name
        )
        return data_connection
    def get_data_validation_config(self)->DataValidationConfig:
        config=self.config[0].data_validation
        schema=self.schema.COLUMNS
        data_validation=DataValidationConfig(
            data_source_path_kaggle=config.data_source_kaggle,
            data_source_path_gdrive=config.data_source_gdrive,
            Status_report=config.STATUS_REPORT_FILE,
            all_schema=schema
        )
        return data_validation

In [11]:
config=ConfigurationManager()
config.get_data_validation_config()

[2025-02-23 17:16:17,134 : INFO: common: yaml file config\config.yaml loaded successfully!]
[2025-02-23 17:16:17,145 : INFO: common: yaml file config\params.yaml loaded successfully!]
[2025-02-23 17:16:17,156 : INFO: common: yaml file config\schema.yaml loaded successfully!]


DataValidationConfig(data_source_path_kaggle='data/interim/kaggle/', data_source_path_gdrive='data/interim/gdrive/', Status_report='docs/data_validation/status_report.txt', all_schema=ConfigBox({'CustomerID': 'int64', 'Tenure': 'float64', 'PreferredLoginDevice': 'object', 'CityTier': 'int64', 'WarehouseToHome': 'float64', 'PreferredPaymentMode': 'object', 'Gender': 'object', 'HourSpendOnApp': 'float64', 'NumberOfDeviceRegistered': 'int64', 'PreferedOrderCat': 'object', 'SatisfactionScore': 'int64', 'MaritalStatus': 'object', 'NumberOfAddress': 'int64', 'Complain': 'int64', 'OrderAmountHikeFromlastYear': 'float64', 'CouponUsed': 'float64', 'OrderCount': 'float64', 'DaySinceLastOrder': 'float64', 'CashbackAmount': 'float64'}))

In [ ]:
import os
import pandas as pd
import kagglehub
import shutil
import gdown
from src.configuration.config import ConfigurationManager
from src.logger import logger
from datetime import datetime

class DataValidation:
    def __init__(self,config=DataValidationConfig):
        self.data_config=config
    def validate_all_columns(self)->bool:
        try:
            validation_status=None
            data=pd.read_csv(self.config)
            

In [14]:
data_valid=DataValidation()
data_valid.validate_all_columns()

True

In [48]:
import great_expectations as ge 
df=pd.read_csv("data/interim/kaggle/raw_kaggle_churn_data.csv")
context=ge.get_context(mode='file')
base_directory = "reports/"  # this is the default path (relative to the root folder of the Data Context) but can be changed as required
site_config = {
    "class_name": "SiteBuilder",
    "site_index_builder": {"class_name": "DefaultSiteIndexBuilder"},
    "store_backend": {
        "class_name": "TupleFilesystemStoreBackend",
        "base_directory": base_directory,
    },
}
site_name = "my_data_docs_site"
context.build_data_docs(site_names=site_name)
context.add_data_docs_site(site_name=site_name, site_config=site_config)

data_source=context.data_sources.add_pandas("pandas")
data_asset=data_source.add_dataframe_asset(name="pd dataframe asset")
batch_defintion=data_asset.add_batch_definition_whole_dataframe("batch_definition")
batch=batch_defintion.get_batch(batch_parameters={"dataframe":df})
expectation=ge.expectations.ExpectColumnValuesToBeBetween(
    column="CityTier",max_value=3,min_value=1
)
validation=batch.validate(expectation,result_format='COMPLETE')

[2025-02-23 16:51:41,174 : INFO: file_data_context: FileDataContext loading fluent config]
[2025-02-23 16:51:41,181 : INFO: config: Loading 'datasources' ->
[]]
[2025-02-23 16:51:41,389 : INFO: file_data_context: Saving 1 Fluent Datasources to c:\Users\asus\Documents\Projects\ECommerce Customer Churn Prediction\gx\great_expectations.yml]
[2025-02-23 16:51:41,393 : INFO: fluent_base_model: PandasDatasource.dict() - missing `config_provider`, skipping config substitution]
[2025-02-23 16:51:41,425 : INFO: file_data_context: Saving 1 Fluent Datasources to c:\Users\asus\Documents\Projects\ECommerce Customer Churn Prediction\gx\great_expectations.yml]
[2025-02-23 16:51:41,429 : INFO: fluent_base_model: DataFrameAsset.dict() - missing `config_provider`, skipping config substitution]
[2025-02-23 16:51:41,431 : INFO: fluent_base_model: PandasDatasource.dict() - missing `config_provider`, skipping config substitution]
[2025-02-23 16:51:41,460 : INFO: file_data_context: Saving 1 Fluent Datasource

Calculating Metrics: 100%|██████████| 10/10 [00:00<00:00, 204.05it/s]


In [44]:
expectation=ge.expectations.ExpectColumnValuesToBeBetween(
    column="CityTier",max_value=3,min_value=1
)
validation=batch.validate(expectation,result_format='COMPLETE')

Calculating Metrics:  30%|███       | 3/10 [00:00<00:00, 333.15it/s]

Calculating Metrics: 100%|██████████| 10/10 [00:00<00:00, 227.27it/s]


In [9]:
pd.read_json(validation)

NameError: name 'pd' is not defined

In [46]:
print(validation)

{
  "success": true,
  "expectation_config": {
    "type": "expect_column_values_to_be_between",
    "kwargs": {
      "batch_id": "pandas-pd dataframe asset",
      "column": "CityTier",
      "min_value": 1.0,
      "max_value": 3.0
    },
    "meta": {}
  },
  "result": {
    "element_count": 2815,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0,
    "partial_unexpected_counts": [],
    "partial_unexpected_index_list": [],
    "unexpected_list": [],
    "unexpected_index_list": [],
    "unexpected_index_query": "df.filter(items=[], axis=0)"
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}


In [47]:
context.open_data_docs()

NoDataDocsError: No Data Docs found. Please check that you have run a checkpoint, and that the checkpoint has a UpdateDataDocsAction in its actions.

In [8]:
import great_expectations as ge

context=ge.get_context()
context.build_data_docs()

validator=context.sources.pandas_default.read_csv("data/interim/kaggle/raw_kaggle_churn_data.csv")

validator.expect_column_values_to_be_between(column="CityTier",max_value=3,min_value=1)

validator.save_expectation_suite(discard_failed_expectations=False)

checkpoint=context.add_or_update_checkpoint(
    name="checkpoint",
    validator=validator,
)
checkpoint_result=checkpoint.run()
context.view_validation_result(checkpoint_result)



Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/10 [00:00<?, ?it/s]

In [1]:
pwd

'c:\\Users\\asus\\Documents\\Projects\\ECommerce Customer Churn Prediction\\notebooks'

In [2]:
import os
os.chdir("../")

In [27]:
import great_expectations as gx

import pandas as pd

df = pd.read_csv(
    "data/interim/kaggle/raw_kaggle_churn_data.csv"
)

context = gx.get_context(context_root_dir="great_expectations")
data_source = context.data_sources.add_pandas("pandas")
data_asset = data_source.add_dataframe_asset(name="pd dataframe asset")


batch_definition = data_asset.add_batch_definition_whole_dataframe("batch definition")
batch = batch_definition.get_batch(batch_parameters={"dataframe": df})

expectation_suite_name="my expectation suite"
#context.add_or_update_expectation_suite(expectation_suite_name=expectation_suite_name)
expectation=gx.expectations.ExpectColumnValuesToBeBetween(
    column="CityTier",max_value=3,min_value=1
)
#batch.add_(expectation)
batch.save_expectation_suite()
validation_result = batch.validate(expectation)
print(validation_result)  # Print the validation result in JSON format

# Build and open data docs
context.build_data_docs()
context.open_data_docs()


c:\Users\asus\anaconda3\envs\venv\Lib\site-packages\great_expectations\data_context\data_context\serializable_data_context.py:160: UserWarning:

Warning. An existing `great_expectations.yml` was found here: c:\Users\asus\Documents\Projects\ECommerce Customer Churn Prediction\gx.
    - No action was taken.

c:\Users\asus\anaconda3\envs\venv\Lib\site-packages\great_expectations\data_context\data_context\serializable_data_context.py:168: UserWarning:

Warning. An existing `config_variables.yml` was found here:
            c:\Users\asus\Documents\Projects\ECommerce Customer Churn Prediction\gx\uncommitted. - No action was taken.



AttributeError: 'Batch' object has no attribute 'save_expectation_suite'

In [14]:
from great_expectations.render.renderer import ValidationResultsPageRenderer
from great_expectations.render.view import DefaultMarkdownPageView
from great_expectations.render.components import RenderedDocumentContent

rendered_content=ValidationResultsPageRenderer().render(validation_results=validation_result)
html_report=DefaultMarkdownPageView().render(RenderedDocumentContent(rendered_content))

with open("data_validation.html","w") as f:
    f.write(html_report)
print("Valdation complete")


KeyError: 'run_id'

In [40]:
import great_expectations as gx
from great_expectations.core.batch import RuntimeBatchRequest
import pandas as pd
from great_expectations import checkpoint

# Load your CSV file into a Pandas DataFrame
df = pd.read_csv("data/interim/kaggle/raw_kaggle_churn_data.csv")

# Initialize the DataContext
context = gx.get_context()

# Add a Pandas datasource
datasource = context.data_sources.add_pandas(name="my_pandas_datasource")

# Create a data asset from the DataFrame
data_asset = datasource.add_dataframe_asset(name="my_dataframe_asset")

# Build a batch request using RuntimeBatchRequest
batch_definition = data_asset.add_batch_definition_whole_dataframe("batch definition")
batch = batch_definition.get_batch(batch_parameters={"dataframe": df})
# Create an expectation suite
expectation_suite_name = "my_expectation_suite"
suite=gx.ExpectationSuite(name=expectation_suite_name)
suite=context.suites.add(suite)

# Get a validator

expectation=gx.expectations.ExpectColumnValuesToBeBetween(
    column="WarehouseToHome", min_value=5, max_value=126
)
# Add expectation for "CityTier" values between 1 and 3
suite.add_expectation(
    gx.expectations.ExpectColumnValuesToBeBetween(
        column="CityTier", min_value=1, max_value=3
    )
)
suite.add_expectation(expectation=expectation)


# Save the expectation suite
expectation.save()

definition_name="my_validation_definition"
validation_definition=gx.ValidationDefinition(
    data=batch_definition,suite=expectation_suite_name,name=definition_name
)
validation_definition = context.validation_definitions.add(validation_definition)
validator = batch.validate(expectation)
# Run validation
checkpoint_result = checkpoint.run()
print(checkpoint_result)  # Print results

# Build and open data docs
context.build_data_docs()
context.open_data_docs()


ValidationError: 1 validation error for ValidationDefinition
suite
  Suite must be a dictionary (if being deserialized) or an ExpectationSuite object. (type=value_error)

In [ ]:
from deepchecks.tabular.suites import data_integrity

integ_suite=data_integrity()
df = pd.read_csv(
    "data/interim/kaggle/raw_kaggle_churn_data.csv"
)
suite_result=integ_suite.run(df)

suite_result.save_as_("reports/validation_report.html")
suite_result.show()

deepchecks - WARNING - Received a "pandas.DataFrame" instance. It is recommended to pass a "deepchecks.tabular.Dataset" instance by initializing it with the data and metadata, for example by doing "Dataset(dataframe, label=label, cat_features=cat_features)"


deepchecks - WARNING - It is recommended to initialize Dataset with categorical features by doing "Dataset(df, cat_features=categorical_list)". No categorical features were passed, therefore heuristically inferring categorical features in the data. 18 categorical features were inferred.: Churn, PreferredLoginDevice, CityTier, PreferredPaymentMode, Gender, HourSpendOnApp, NumberOfDeviceRegistered... For full list use dataset.cat_features


c:\Users\asus\anaconda3\envs\venv\Lib\site-packages\deepchecks\tabular\suite.py:117: DeprecationWarning:

the `interpolation=` argument to quantile was renamed to `method=`, which has additional options.
Users of the modes 'nearest', 'lower', 'higher', or 'midpoint' are encouraged to review the method they used. (Deprecated NumPy 1.22)

c:\Users\asus\anaconda3\envs\venv\Lib\site-packages\deepchecks\utils\correlation_methods.py:102: RuntimeWarning:

invalid value encountered in scalar divide

c:\Users\asus\anaconda3\envs\venv\Lib\site-packages\_plotly_utils\basevalidators.py:2596: DeprecationWarning:

*scattermapbox* is deprecated! Use *scattermap* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/

c:\Users\asus\anaconda3\envs\venv\Lib\site-packages\_plotly_utils\basevalidators.py:2596: DeprecationWarning:

*scattermapbox* is deprecated! Use *scattermap* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/

c:\Users\asus\anaconda3\envs\venv\Lib\site-

Accordion(children=(VBox(children=(HTML(value='\n<h1 id="summary_906GRD7IH42XXJQM7NUD4I2I9">Data Integrity Sui…

In [ ]:
import great_expectations as ge 
from great_expectations.core.batch import RuntimeBatchRequest
df = pd.read_csv(
    "data/interim/kaggle/raw_kaggle_churn_data.csv"
)

context=ge.get_context()


context.add_datasource(**datasource_config)

DataContextError: Datasource is not a FluentDatasource

In [ ]:
import great_expectations as gx
from great_expectations import expectations as gxe
import pandas as pd
from great_expectations.checkpoint import UpdateDataDocsAction

#generating context
context=gx.get_context(mode='file')


df = pd.read_csv(
    "data/interim/kaggle/raw_kaggle_churn_data.csv"
)

#giving a data source name to identify all data sources uniquely
datasource = context.data_sources.add_pandas(name="my_pandas_datasource")

# Create a data asset from the DataFrame
data_asset = datasource.add_dataframe_asset(name="my_dataframe_asset")

# Build a batch request using RuntimeBatchRequest
batch_definition = data_asset.add_batch_definition_whole_dataframe("batch definition")
batch = batch_definition.get_batch(batch_parameters={"dataframe": df})

#Organize expectation suite
suite_name="my_expectation_suite"
suite=gx.ExpectationSuite(name=suite_name)
suite=context.suites.add(suite)
expectation=gx.expectations.ExpectColumnValuesToBeBetween(
    column="WarehouseToHome", min_value=5, max_value=126
)
suite.add_expectation(expectation)
expectation.save()
# validation definition name
validation_definition_name="dataset"
validation_definition=gx.ValidationDefinition(
    data=batch_definition, suite=suite,name=validation_definition_name
)
validation=context.validation_definitions.add(validation_definition)
#validation_result=validation.run(batch_parameters={"dataframe":df})
#print(validation_result)

# Define actions
action_list=[
    UpdateDataDocsAction(
        name="update_all_data_docs"
    )
]
#create checkpoint
validation_defintions=[context.validation_definitions.get("dataset")]
checkpoint_name="first check" 
checkpoint=gx.Checkpoint(
    name=checkpoint_name,
    validation_definitions=validation_defintions,
    actions=action_list,
    result_format={"result_format":"COMPLETE"}
)

#save the checkpoint
context.checkpoints.add(checkpoint)

validation_result=checkpoint.run(
    batch_parameters={"dataframe":df}
)


#Configure Data Docs
base_directory = "uncommitted/data_docs/local_site/"  # this is the default path (relative to the root folder of the Data Context) but can be changed as required
site_config = {
    "class_name": "SiteBuilder",
    "site_index_builder": {"class_name": "DefaultSiteIndexBuilder"},
    "store_backend": {
        "class_name": "TupleFilesystemStoreBackend",
        "base_directory": base_directory,
    },
}
site_name="my_data_docs_site"

context.add_data_docs_site(site_name=site_name,site_config=site_config)

context.build_data_docs(site_names=site_name)

context.open_data_docs()



Calculating Metrics:   0%|          | 0/10 [00:00<?, ?it/s]

In [3]:
pwd

'c:\\Users\\asus\\Documents\\Projects\\ECommerce Customer Churn Prediction'

In [2]:
import os
os.chdir("../")

In [ ]:
import great_expectations as gx
from great_expectations import expectations as gxe
import pandas as pd
from great_expectations.checkpoint import UpdateDataDocsAction
from src.utils.common import read_yaml
from src.constants import CONFIG_PATH,PARAMS_PATH,SCHEMA_PATH







In [ ]:
from src.constants import CONFIG_PATH,PARAMS_PATH,SCHEMA_PATH
from src.utils.common import read_yaml
from src.entity.data_ingestion import DataIngestionConfig
from src.entity.data_storage import DataStorageConnectionConfig
from src.entity.data_validation import DataValidationConfig




class ConfigurationManager:
    def __init__(self,config_path=CONFIG_PATH,params_path=PARAMS_PATH,schema_path=SCHEMA_PATH):
        self.config=read_yaml(config_path),
        self.params=read_yaml(params_path),
        self.schema=read_yaml(schema_path)
    def get_data_ingestion_config(self)->DataIngestionConfig:
        config=self.config[0].data
        data_ingestion=DataIngestionConfig(
            kaggle_dataset=config.kaggle_source_path,
            gdrive_dataset=config.gdrive_source_path,
            local_dataset=config.raw,
            staging_dataset=config.interim
        )
        return data_ingestion
    def get_data_storage_config(self)->DataStorageConnectionConfig:
        config=self.config[0].sql
        data_connection=DataStorageConnectionConfig(
            server_name=config.server_name,
            db_name=config.db_name,
            driver_name=config.driver_name,
            trusted_conn=config.trusted_conn,
            kaggle_table=config.kaggle_table_name,
            gdrive_table=config.gdrive_table_name
        )
        return data_connection
    def get_data_validation_config(self)->DataValidationConfig:
        config=self.config[0].data_validation,
        schema=self.schema.COLUMNS
        print(config)
        data_validation=DataValidationConfig(
            data_source_path_kaggle=config[0].data_source_kaggle,
            data_source_path_gdrive=config[0].data_source_gdrive,
            Status_report=config[0].STATUS_REPORT_FILE,
            all_schema=schema
        )
        return data_validation
    
   

In [100]:
import shutil
class DataValidation:
    def __init__(self):
        self.config=ConfigurationManager().get_data_validation_config()
    def run_validation(self):
        #Create a context
        if(os.path.exists("gx/") and os.path.isdir("gx/")):
            shutil.rmtree("gx/")
        context=gx.get_context(mode='file')
        #read data into dataframe
        df_kaggle = pd.read_csv(self.config.data_source_path_kaggle+"raw_kaggle_churn_data.csv")
        df_gdrive = pd.read_csv(self.config.data_source_path_gdrive+"raw_gdrive_churn_data.csv")
        combined_df=pd.concat([df_kaggle,df_gdrive],axis=0)
        #giving a data source name to identify all data sources uniquely
        datasource = context.data_sources.add_pandas(name="churn_data_source")

        # Create a data asset from the DataFrame
        data_asset = datasource.add_dataframe_asset(name="churn_data_asset")

        # Build a batch request using whole dataframe
        batch_definition = data_asset.add_batch_definition_whole_dataframe("bulk batch")
        batch = batch_definition.get_batch(batch_parameters={"dataframe": combined_df})
        
        #Organize expectation suite
        col_existence_suite_name="Column existence validation suite"
        col_suite=gx.ExpectationSuite(name=col_existence_suite_name)
        
        is_datatype_same_suite="Data Type Check Suite"
        type_suite=gx.ExpectationSuite(name=is_datatype_same_suite)
        suite_1=context.suites.add(col_suite)
        suite_2=context.suites.add(type_suite)
        # expectation=gx.expectations.ExpectColumnValuesToBeBetween(
        #     column="WarehouseToHome", min_value=5, max_value=126
        # )
        schema=self.config.all_schema
        for col in schema.keys():
                expectation1=gx.expectations.ExpectColumnToExist(column=col)
                suite_1.add_expectation(expectation1)
        expectation1.save()
        for col,dtype in schema.items():
            try:
                expectation2=gx.expectations.ExpectColumnValuesToBeOfType(column=col,type_=dtype)
                suite_2.add_expectation(expectation2)
            except Exception as e:
                raise e
                
            
        #suite.add_expectation(expectation)
        expectation2.save()
        # validation definition name
        validation_definition_name_col="dataset_cols"
        validation_definition_cols=gx.ValidationDefinition(
            data=batch_definition, suite=suite_1,name=validation_definition_name_col
        )
        validation_definition_name_type="dataset_types"
        validation_definition_types=gx.ValidationDefinition(
            data=batch_definition, suite=suite_2,name=validation_definition_name_type
        )
        context.validation_definitions.add(validation_definition_cols)
        context.validation_definitions.add(validation_definition_types)
        action_list=[
        UpdateDataDocsAction(
        name="update_all_data_docs"
         )
        ]
        #create checkpoint
        validation_defintions=[context.validation_definitions.get("dataset_types"),
        context.validation_definitions.get("dataset_cols")]
        checkpoint_name="first check" 
        checkpoint=gx.Checkpoint(
            name=checkpoint_name,
            validation_definitions=validation_defintions,
            actions=action_list,
            result_format={"result_format":"COMPLETE"}
        )

        #save the checkpoint
        context.checkpoints.add(checkpoint)

        validation_result=checkpoint.run(
            batch_parameters={"dataframe":combined_df}
        )


        #Configure Data Docs
        base_directory = "uncommitted/data_docs/local_site/"  # this is the default path (relative to the root folder of the Data Context) but can be changed as required
        site_config = {
            "class_name": "SiteBuilder",
            "site_index_builder": {"class_name": "DefaultSiteIndexBuilder"},
            "store_backend": {
                "class_name": "TupleFilesystemStoreBackend",
                "base_directory": base_directory,
            },
        }
        site_name="my_data_docs_site"

        context.add_data_docs_site(site_name=site_name,site_config=site_config)

        context.build_data_docs(site_names=site_name)

        return True
        
        
        
        
        
        
        

In [101]:
test=DataValidation()
test.run_validation()

[2025-02-24 00:46:39,478 : INFO: common: yaml file config\config.yaml loaded successfully!]
[2025-02-24 00:46:39,485 : INFO: common: yaml file config\params.yaml loaded successfully!]
[2025-02-24 00:46:39,497 : INFO: common: yaml file config\schema.yaml loaded successfully!]
(ConfigBox({'data_source_kaggle': 'data/interim/kaggle/', 'data_source_gdrive': 'data/interim/gdrive/', 'STATUS_REPORT_FILE': 'uncommitted/data_docs/local_site/'}),)
[2025-02-24 00:46:39,589 : INFO: file_data_context: FileDataContext loading fluent config]
[2025-02-24 00:46:39,600 : INFO: config: Loading 'datasources' ->
[]]
[2025-02-24 00:46:39,767 : INFO: file_data_context: Saving 1 Fluent Datasources to c:\Users\asus\Documents\Projects\ECommerce Customer Churn Prediction\gx\great_expectations.yml]
[2025-02-24 00:46:39,770 : INFO: fluent_base_model: PandasDatasource.dict() - missing `config_provider`, skipping config substitution]
[2025-02-24 00:46:39,815 : INFO: file_data_context: Saving 1 Fluent Datasources to 

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

[2025-02-24 00:46:44,027 : INFO: expectation: _get_default_value called with key "table", but it is not a known field]
[2025-02-24 00:46:44,031 : INFO: expectation: _get_default_value called with key "table", but it is not a known field]
[2025-02-24 00:46:44,037 : INFO: expectation: _get_default_value called with key "table", but it is not a known field]
[2025-02-24 00:46:44,043 : INFO: expectation: _get_default_value called with key "table", but it is not a known field]
[2025-02-24 00:46:44,051 : INFO: expectation: _get_default_value called with key "table", but it is not a known field]
[2025-02-24 00:46:44,058 : INFO: expectation: _get_default_value called with key "table", but it is not a known field]
[2025-02-24 00:46:44,063 : INFO: expectation: _get_default_value called with key "table", but it is not a known field]
[2025-02-24 00:46:44,069 : INFO: expectation: _get_default_value called with key "table", but it is not a known field]
[2025-02-24 00:46:44,076 : INFO: expectation: _g

Calculating Metrics:   0%|          | 0/2 [00:00<?, ?it/s]

[2025-02-24 00:46:44,433 : INFO: expectation: _get_default_value called with key "table", but it is not a known field]
[2025-02-24 00:46:44,439 : INFO: expectation: _get_default_value called with key "table", but it is not a known field]
[2025-02-24 00:46:44,442 : INFO: expectation: _get_default_value called with key "table", but it is not a known field]
[2025-02-24 00:46:44,445 : INFO: expectation: _get_default_value called with key "table", but it is not a known field]
[2025-02-24 00:46:44,450 : INFO: expectation: _get_default_value called with key "table", but it is not a known field]
[2025-02-24 00:46:44,457 : INFO: expectation: _get_default_value called with key "table", but it is not a known field]
[2025-02-24 00:46:44,463 : INFO: expectation: _get_default_value called with key "table", but it is not a known field]
[2025-02-24 00:46:44,471 : INFO: expectation: _get_default_value called with key "table", but it is not a known field]
[2025-02-24 00:46:44,478 : INFO: expectation: _g

True

[2025-02-24 00:47:16,464 : INFO: _common: Backing off send_request(...) for 0.2s (requests.exceptions.ReadTimeout: HTTPSConnectionPool(host='posthog.greatexpectations.io', port=443): Read timed out. (read timeout=15))]
[2025-02-24 00:47:46,425 : INFO: _common: Backing off send_request(...) for 0.9s (requests.exceptions.ReadTimeout: HTTPSConnectionPool(host='posthog.greatexpectations.io', port=443): Read timed out. (read timeout=15))]
[2025-02-24 00:48:17,533 : INFO: _common: Backing off send_request(...) for 2.3s (requests.exceptions.ReadTimeout: HTTPSConnectionPool(host='posthog.greatexpectations.io', port=443): Read timed out. (read timeout=15))]


In [ ]:
df_kaggle = pd.read_csv("data/interim/kaggle/raw_kaggle_churn_data.csv")
df_gdrive = pd.read_csv("data/interim/gdrive/raw_gdrive_churn_data.csv")
combined_df=pd.concat([df_kaggle,df_gdrive],axis=0)
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5630 entries, 0 to 2814
Data columns (total 22 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   CustomerID                   5630 non-null   int64  
 1   Churn                        5630 non-null   int64  
 2   Tenure                       5366 non-null   float64
 3   PreferredLoginDevice         5630 non-null   object 
 4   CityTier                     5630 non-null   int64  
 5   WarehouseToHome              5379 non-null   float64
 6   PreferredPaymentMode         5630 non-null   object 
 7   Gender                       5630 non-null   object 
 8   HourSpendOnApp               5375 non-null   float64
 9   NumberOfDeviceRegistered     5630 non-null   int64  
 10  PreferedOrderCat             5630 non-null   object 
 11  SatisfactionScore            5630 non-null   int64  
 12  MaritalStatus                5630 non-null   object 
 13  NumberOfAddress        

[2025-02-23 23:18:00,508 : INFO: _common: Backing off send_request(...) for 2.7s (requests.exceptions.ReadTimeout: HTTPSConnectionPool(host='posthog.greatexpectations.io', port=443): Read timed out. (read timeout=15))]
[2025-02-23 23:18:34,806 : ERROR: _common: Giving up send_request(...) after 4 tries (requests.exceptions.ReadTimeout: HTTPSConnectionPool(host='posthog.greatexpectations.io', port=443): Read timed out. (read timeout=15))]
[2025-02-23 23:18:34,808 : ERROR: consumer: error uploading: HTTPSConnectionPool(host='posthog.greatexpectations.io', port=443): Read timed out. (read timeout=15)]
